In [130]:
# Import dependencies
import json
import pandas as pd
import numpy as np
import re

In [2]:
# Build a file path to the directory holding the data.
file_dir = 'C://Users/maghs/Git/Movies-ETL/Data'

## Wikipedia Data on Movies 1990 - 2018

In [3]:
# Open a file in directory using f-string inseated of typing the whole directory every time, update file_dir.
# f'{file_dir}/filename'

# Load the raw JSON into a list of dictionaries.
with open(f'{file_dir}/wikipedia-movies.json', mode='r') as file:
    wiki_movies_raw = json.load(file)

In [4]:
# Check how many records were loaded from JSON.
len(wiki_movies_raw)

7311

In [5]:
# First 5 records.
wiki_movies_raw[:5]

[{'url': 'https://en.wikipedia.org/wiki/The_Adventures_of_Ford_Fairlane',
  'year': 1990,
  'imdb_link': 'https://www.imdb.com/title/tt0098987/',
  'title': 'The Adventures of Ford Fairlane',
  'Directed by': 'Renny Harlin',
  'Produced by': ['Steve Perry', 'Joel Silver'],
  'Screenplay by': ['David Arnott', 'James Cappe', 'Daniel Waters'],
  'Story by': ['David Arnott', 'James Cappe'],
  'Based on': ['Characters', 'by Rex Weiner'],
  'Starring': ['Andrew Dice Clay',
   'Wayne Newton',
   'Priscilla Presley',
   'Lauren Holly',
   'Morris Day',
   'Robert Englund',
   "Ed O'Neill"],
  'Narrated by': 'Andrew "Dice" Clay',
  'Music by': ['Cliff Eidelman', 'Yello'],
  'Cinematography': 'Oliver Wood',
  'Edited by': 'Michael Tronick',
  'Productioncompany ': 'Silver Pictures',
  'Distributed by': '20th Century Fox',
  'Release date': ['July 11, 1990', '(', '1990-07-11', ')'],
  'Running time': '102 minutes',
  'Country': 'United States',
  'Language': 'English',
  'Budget': '$20 million',


In [6]:
# Last 5 records. 
wiki_movies_raw[-5:]

[{'url': 'https://en.wikipedia.org/wiki/Holmes_%26_Watson',
  'year': 2018,
  'imdb_link': 'https://www.imdb.com/title/tt1255919/',
  'title': 'Holmes & Watson',
  'Directed by': 'Etan Cohen',
  'Produced by': ['Will Ferrell',
   'Adam McKay',
   'Jimmy Miller',
   'Clayton Townsend'],
  'Screenplay by': 'Etan Cohen',
  'Based on': ['Sherlock Holmes',
   'and',
   'Dr. Watson',
   'by',
   'Sir Arthur Conan Doyle'],
  'Starring': ['Will Ferrell',
   'John C. Reilly',
   'Rebecca Hall',
   'Rob Brydon',
   'Steve Coogan',
   'Ralph Fiennes'],
  'Music by': 'Mark Mothersbaugh',
  'Cinematography': 'Oliver Wood',
  'Edited by': 'Dean Zimmerman',
  'Productioncompanies ': ['Columbia Pictures',
   'Gary Sanchez Productions',
   'Mosaic Media Group',
   'Mimran Schur Pictures'],
  'Distributed by': 'Sony Pictures Releasing',
  'Release date': ['December 25, 2018',
   '(',
   '2018-12-25',
   ')',
   '(United States)'],
  'Running time': '90 minutes',
  'Country': 'United States',
  'Language

In [7]:
# Check some records in the middle.
wiki_movies_raw[3600:3605]

[{'url': 'https://en.wikipedia.org/wiki/Benji:_Off_the_Leash!',
  'year': 2004,
  'imdb_link': 'https://www.imdb.com/title/tt0315273/',
  'title': 'Benji: Off the Leash!',
  'Directed by': 'Joe Camp',
  'Written by': 'Joe Camp',
  'Starring': ['Benji', 'Nick Whitaker', 'Shaggy', 'Gypsy the Cockatoo'],
  'Music by': 'Antonio di Lorenzo',
  'Productioncompany ': 'Mulberry Square Productions',
  'Distributed by': 'Mulberry Square Productions',
  'Release date': ['March 26, 2004', '(', '2004-03-26', ')'],
  'Running time': '97 min',
  'Country': 'United States',
  'Language': 'English',
  'Box office': '$3,817,362'},
 {'url': 'https://en.wikipedia.org/wiki/The_Best_Thief_in_the_World',
  'year': 2004,
  'imdb_link': 'https://www.imdb.com/title/tt0389796/',
  'title': 'The Best Thief in the World',
  'Directed by': 'Jacob Kornbluth',
  'Produced by': ['Tim Perrell', 'Nicola Usborne'],
  'Written by': 'Jacob Kornbluth',
  'Starring': ['Marc Rozendaal',
   'Michael Silverman',
   'David Warsh

## MovieLens Ratings & Metadata 

In [8]:
# Read the Kaggle CSV data.
kaggle_metadata = pd.read_csv(f'{file_dir}/movies_metadata.csv', low_memory=False)
ratings = pd.read_csv(f'{file_dir}/ratings.csv')

In [9]:
kaggle_metadata.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [10]:
kaggle_metadata.tail()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
45461,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",http://www.imdb.com/title/tt6209470/,439050,tt6209470,fa,رگ خواب,Rising and falling between a man and woman.,...,NaN,0.0,90.0,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Released,Rising and falling between a man and woman,Subdue,False,4.0,1.0
45462,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,...,2011-11-17,0.0,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,False,9.0,3.0
45463,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",...,2003-08-01,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,False,3.8,6.0
45464,False,NaN,0,[],NaN,227506,tt0008536,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",...,1917-10-21,0.0,87.0,[],Released,NaN,Satan Triumphant,False,0.0,0.0
45465,False,NaN,0,[],NaN,461257,tt6980792,en,Queerama,50 years after decriminalisation of homosexual...,...,2017-06-09,0.0,75.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Queerama,False,0.0,0.0


In [11]:
kaggle_metadata.sample(5)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
5060,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10402, 'n...",NaN,15310,tt0080948,en,The Jazz Singer,Neil Diamond stars as Yussel in this tale of a...,...,1980-12-17,0.0,115.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,His story will make you cry. His music will ma...,The Jazz Singer,False,5.9,10.0
32079,False,"{'id': 41083, 'name': 'Scuola di ladri - Colle...",0,[],NaN,38313,tt0091911,it,Scuola di ladri,Frenetic in its pacing and with miniscule char...,...,1986-09-25,0.0,96.0,"[{'iso_639_1': 'it', 'name': 'Italiano'}]",Released,NaN,School of Thieves,False,5.9,51.0
4804,False,"{'id': 284024, 'name': 'The Exterminator Colle...",0,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",NaN,37835,tt0080707,en,The Exterminator,A man's best friend is killed on the streets o...,...,1980-09-10,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A one man army. A new kind of soldier in a new...,The Exterminator,False,6.1,35.0
161,False,NaN,27000000,"[{'id': 18, 'name': 'Drama'}, {'id': 9648, 'na...",NaN,8512,tt0112857,en,Devil in a Blue Dress,"In late 1940s Los Angeles, Easy Rawlins is an ...",...,1995-09-15,16140822.0,102.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Private detective Easy Rawlins has been caught...,Devil in a Blue Dress,False,6.3,88.0
41992,False,NaN,0,"[{'id': 35, 'name': 'Comedy'}]",NaN,365037,tt3599642,en,Swearnet Live,"""Trailer Park Boys"" John Paul Tremblay, Robb W...",...,2014-10-01,0.0,75.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Swearnet Live,False,6.0,2.0


In [12]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,110,1.0,1425941529
1,1,147,4.5,1425942435
2,1,858,5.0,1425941523
3,1,1221,5.0,1425941546
4,1,1246,5.0,1425941556


In [13]:
ratings.tail()

,userId,movieId,rating,timestamp
26024284,270896,58559,5.0,1257031564
26024285,270896,60069,5.0,1257032032
26024286,270896,63082,4.5,1257031764
26024287,270896,64957,4.5,1257033990
26024288,270896,71878,2.0,1257031858


In [14]:
ratings.sample(n=5)

,userId,movieId,rating,timestamp
6471348,66780,2571,5.0,1457649211
6757923,69741,2115,3.5,1391135949
10600535,109389,3969,3.5,1119187713
22179435,230241,8464,4.0,1134416243
7891825,81353,3910,3.5,1201099232


## Investigate the Wikipedia Data

In [15]:
# Turn the raw wiki data into a DataFrame.
wiki_movies_df = pd.DataFrame(wiki_movies_raw)
wiki_movies_df.head()

,url,year,imdb_link,title,Directed by,Produced by,Screenplay by,Story by,Based on,Starring,...,Predecessor,Founders,Area served,Products,Services,Russian,Hebrew,Revenue,Operating income,Polish
0,https://en.wikipedia.org/wiki/The_Adventures_o...,1990.0,https://www.imdb.com/title/tt0098987/,The Adventures of Ford Fairlane,Renny Harlin,"[Steve Perry, Joel Silver]","[David Arnott, James Cappe, Daniel Waters]","[David Arnott, James Cappe]","[Characters, by Rex Weiner]","[Andrew Dice Clay, Wayne Newton, Priscilla Pre...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"https://en.wikipedia.org/wiki/After_Dark,_My_S...",1990.0,https://www.imdb.com/title/tt0098994/,"After Dark, My Sweet",James Foley,"[Ric Kidney, Robert Redlin]","[James Foley, Robert Redlin]",NaN,"[the novel, After Dark, My Sweet, by, Jim Thom...","[Jason Patric, Rachel Ward, Bruce Dern, George...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://en.wikipedia.org/wiki/Air_America_(film),1990.0,https://www.imdb.com/title/tt0099005/,Air America,Roger Spottiswoode,Daniel Melnick,"[John Eskow, Richard Rush]",NaN,"[Air America, by, Christopher Robbins]","[Mel Gibson, Robert Downey Jr., Nancy Travis, ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://en.wikipedia.org/wiki/Alice_(1990_film),1990.0,https://www.imdb.com/title/tt0099012/,Alice,Woody Allen,Robert Greenhut,NaN,NaN,NaN,"[Alec Baldwin, Blythe Danner, Judy Davis, Mia ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://en.wikipedia.org/wiki/Almost_an_Angel,1990.0,https://www.imdb.com/title/tt0099018/,Almost an Angel,John Cornell,John Cornell,NaN,NaN,NaN,"[Paul Hogan, Elias Koteas, Linda Kozlowski]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
# Get the columns of the wiki data by turning them into a list (too many columns to print)
wiki_movies_df.columns.tolist()

['url',
 'year',
 'imdb_link',
 'title',
 'Directed by',
 'Produced by',
 'Screenplay by',
 'Story by',
 'Based on',
 'Starring',
 'Narrated by',
 'Music by',
 'Cinematography',
 'Edited by',
 'Productioncompany ',
 'Distributed by',
 'Release date',
 'Running time',
 'Country',
 'Language',
 'Budget',
 'Box office',
 'Written by',
 'Genre',
 'Theme music composer',
 'Country of origin',
 'Original language(s)',
 'Producer(s)',
 'Editor(s)',
 'Production company(s)',
 'Original network',
 'Original release',
 'Productioncompanies ',
 'Executive producer(s)',
 'Production location(s)',
 'Distributor',
 'Picture format',
 'Audio format',
 'Voices of',
 'Followed by',
 'Composer(s)',
 'Created by',
 'Also known as',
 'Opening theme',
 'No. of episodes',
 'Preceded by',
 'Author',
 'Publisher',
 'Publication date',
 'Media type',
 'Pages',
 'ISBN',
 'OCLC',
 'LC Class',
 'Cover artist',
 'Series',
 'Set in',
 'Adaptation by',
 'Suggested by',
 'Biographical data',
 'Born',
 'Died',
 'Resti

In [17]:
# Filter(1) the data to movies with a director and an IMDB link only, using list comprehenstion.
wiki_movies = [movie for movie in wiki_movies_raw
              if ('Director' in movie or 'Directed by' in movie)
                  and 'imdb_link' in movie]
len(wiki_movies)

7080

In [18]:
# Create a DataFrame from the filtered(1) list.
wiki_movies_df = pd.DataFrame(wiki_movies)
wiki_movies_df.head()

,url,year,imdb_link,title,Directed by,Produced by,Screenplay by,Story by,Based on,Starring,...,Hepburn,Literally,Cantonese,Chinese,Yiddish,Arabic,Romanized,Russian,Hebrew,Polish
0,https://en.wikipedia.org/wiki/The_Adventures_o...,1990,https://www.imdb.com/title/tt0098987/,The Adventures of Ford Fairlane,Renny Harlin,"[Steve Perry, Joel Silver]","[David Arnott, James Cappe, Daniel Waters]","[David Arnott, James Cappe]","[Characters, by Rex Weiner]","[Andrew Dice Clay, Wayne Newton, Priscilla Pre...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"https://en.wikipedia.org/wiki/After_Dark,_My_S...",1990,https://www.imdb.com/title/tt0098994/,"After Dark, My Sweet",James Foley,"[Ric Kidney, Robert Redlin]","[James Foley, Robert Redlin]",NaN,"[the novel, After Dark, My Sweet, by, Jim Thom...","[Jason Patric, Rachel Ward, Bruce Dern, George...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://en.wikipedia.org/wiki/Air_America_(film),1990,https://www.imdb.com/title/tt0099005/,Air America,Roger Spottiswoode,Daniel Melnick,"[John Eskow, Richard Rush]",NaN,"[Air America, by, Christopher Robbins]","[Mel Gibson, Robert Downey Jr., Nancy Travis, ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://en.wikipedia.org/wiki/Alice_(1990_film),1990,https://www.imdb.com/title/tt0099012/,Alice,Woody Allen,Robert Greenhut,NaN,NaN,NaN,"[Alec Baldwin, Blythe Danner, Judy Davis, Mia ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://en.wikipedia.org/wiki/Almost_an_Angel,1990,https://www.imdb.com/title/tt0099018/,Almost an Angel,John Cornell,John Cornell,NaN,NaN,NaN,"[Paul Hogan, Elias Koteas, Linda Kozlowski]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
# Get the columns in the filtered(1) DF.
wiki_movies_df.columns.tolist()

['url',
 'year',
 'imdb_link',
 'title',
 'Directed by',
 'Produced by',
 'Screenplay by',
 'Story by',
 'Based on',
 'Starring',
 'Narrated by',
 'Music by',
 'Cinematography',
 'Edited by',
 'Productioncompany ',
 'Distributed by',
 'Release date',
 'Running time',
 'Country',
 'Language',
 'Budget',
 'Box office',
 'Written by',
 'Genre',
 'Theme music composer',
 'Country of origin',
 'Original language(s)',
 'Producer(s)',
 'Editor(s)',
 'Production company(s)',
 'Original network',
 'Original release',
 'Productioncompanies ',
 'Executive producer(s)',
 'Production location(s)',
 'Distributor',
 'Picture format',
 'Audio format',
 'Voices of',
 'Followed by',
 'Composer(s)',
 'Created by',
 'Also known as',
 'Opening theme',
 'No. of episodes',
 'Preceded by',
 'Adaptation by',
 'Suggested by',
 'Traditional',
 'Mandarin',
 'Released',
 'Recorded',
 'Venue',
 'Length',
 'Label',
 'Director',
 'Producer',
 'Animation by',
 'Color process',
 'Simplified',
 'French',
 'Camera setup'

In [20]:
# second cycle is to clear no. of episode column that belong to TV shows, or add the filter(2) to the first filter code block.
wiki_movies = [movie for movie in wiki_movies_raw
              if ('Director' in movie or 'Directed by' in movie)
                  and 'imdb_link' in movie
                  and 'No. of episodes' not in movie]
len(wiki_movies)    

7076

In [21]:
# Recreate a DataFrame from the filtered(2) list.
wiki_movies_df = pd.DataFrame(wiki_movies)
wiki_movies_df.head()

,url,year,imdb_link,title,Directed by,Produced by,Screenplay by,Story by,Based on,Starring,...,Hepburn,Literally,Cantonese,Chinese,Yiddish,Arabic,Romanized,Russian,Hebrew,Polish
0,https://en.wikipedia.org/wiki/The_Adventures_o...,1990,https://www.imdb.com/title/tt0098987/,The Adventures of Ford Fairlane,Renny Harlin,"[Steve Perry, Joel Silver]","[David Arnott, James Cappe, Daniel Waters]","[David Arnott, James Cappe]","[Characters, by Rex Weiner]","[Andrew Dice Clay, Wayne Newton, Priscilla Pre...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"https://en.wikipedia.org/wiki/After_Dark,_My_S...",1990,https://www.imdb.com/title/tt0098994/,"After Dark, My Sweet",James Foley,"[Ric Kidney, Robert Redlin]","[James Foley, Robert Redlin]",NaN,"[the novel, After Dark, My Sweet, by, Jim Thom...","[Jason Patric, Rachel Ward, Bruce Dern, George...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://en.wikipedia.org/wiki/Air_America_(film),1990,https://www.imdb.com/title/tt0099005/,Air America,Roger Spottiswoode,Daniel Melnick,"[John Eskow, Richard Rush]",NaN,"[Air America, by, Christopher Robbins]","[Mel Gibson, Robert Downey Jr., Nancy Travis, ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://en.wikipedia.org/wiki/Alice_(1990_film),1990,https://www.imdb.com/title/tt0099012/,Alice,Woody Allen,Robert Greenhut,NaN,NaN,NaN,"[Alec Baldwin, Blythe Danner, Judy Davis, Mia ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://en.wikipedia.org/wiki/Almost_an_Angel,1990,https://www.imdb.com/title/tt0099018/,Almost an Angel,John Cornell,John Cornell,NaN,NaN,NaN,"[Paul Hogan, Elias Koteas, Linda Kozlowski]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
# Get the columns in the filtered(2) DF.
wiki_movies_df.columns.tolist()

['url',
 'year',
 'imdb_link',
 'title',
 'Directed by',
 'Produced by',
 'Screenplay by',
 'Story by',
 'Based on',
 'Starring',
 'Narrated by',
 'Music by',
 'Cinematography',
 'Edited by',
 'Productioncompany ',
 'Distributed by',
 'Release date',
 'Running time',
 'Country',
 'Language',
 'Budget',
 'Box office',
 'Written by',
 'Genre',
 'Theme music composer',
 'Country of origin',
 'Original language(s)',
 'Producer(s)',
 'Editor(s)',
 'Production company(s)',
 'Original network',
 'Original release',
 'Productioncompanies ',
 'Executive producer(s)',
 'Production location(s)',
 'Distributor',
 'Picture format',
 'Audio format',
 'Voices of',
 'Followed by',
 'Composer(s)',
 'Created by',
 'Preceded by',
 'Adaptation by',
 'Suggested by',
 'Traditional',
 'Mandarin',
 'Released',
 'Recorded',
 'Venue',
 'Length',
 'Label',
 'Director',
 'Producer',
 'Also known as',
 'Animation by',
 'Color process',
 'Simplified',
 'French',
 'Screen story by',
 'Hangul',
 'Revised Romanization

In [ ]:
# # Create a function to clean each entry
# def clean_movie(movie):
#     movie = dict(movie) # Create a non-destructive copy
#     return movie

In [23]:
wiki_movies_df[wiki_movies_df['Arabic'].notnull()]

,url,year,imdb_link,title,Directed by,Produced by,Screenplay by,Story by,Based on,Starring,...,Hepburn,Literally,Cantonese,Chinese,Yiddish,Arabic,Romanized,Russian,Hebrew,Polish
6834,https://en.wikipedia.org/wiki/The_Insult_(film),2018,https://www.imdb.com/title/tt7048622/,The Insult,Ziad Doueiri,"[Rachid Bouchareb, Jean Bréhat, Julie Gayet, A...",NaN,NaN,NaN,"[Adel Karam, Kamel El Basha]",...,NaN,Case No. 23,NaN,NaN,NaN,قضية رقم ٢٣,Qadiyya raqm 23,NaN,NaN,NaN
7058,https://en.wikipedia.org/wiki/Capernaum_(film),2018,https://www.imdb.com/title/tt8267604/,Capernaum,Nadine Labaki,"[Michel Merkt, Khaled Mouzanar]","[Nadine Labaki, Jihad Hojaily, Michelle Keserw...","[Georges Khabbaz, Nadine Labaki, Michelle Kese...",NaN,"[Zain Al Rafeea, Yordanos Shiferaw, Boluwatife...",...,NaN,NaN,NaN,NaN,NaN,کفرناحوم‎,NaN,NaN,NaN,NaN


In [24]:
wiki_movies_df[wiki_movies_df['Arabic'].notnull()]['url']

6834    https://en.wikipedia.org/wiki/The_Insult_(film)
7058     https://en.wikipedia.org/wiki/Capernaum_(film)
Name: url, dtype: object

In [25]:
# Sort columns in alphabetical order to understand what they contain.
sorted(wiki_movies_df.columns.tolist())

['Adaptation by',
 'Also known as',
 'Animation by',
 'Arabic',
 'Audio format',
 'Based on',
 'Box office',
 'Budget',
 'Cantonese',
 'Chinese',
 'Cinematography',
 'Color process',
 'Composer(s)',
 'Country',
 'Country of origin',
 'Created by',
 'Directed by',
 'Director',
 'Distributed by',
 'Distributor',
 'Edited by',
 'Editor(s)',
 'Executive producer(s)',
 'Followed by',
 'French',
 'Genre',
 'Hangul',
 'Hebrew',
 'Hepburn',
 'Japanese',
 'Label',
 'Language',
 'Length',
 'Literally',
 'Mandarin',
 'McCune–Reischauer',
 'Music by',
 'Narrated by',
 'Original language(s)',
 'Original network',
 'Original release',
 'Original title',
 'Picture format',
 'Polish',
 'Preceded by',
 'Produced by',
 'Producer',
 'Producer(s)',
 'Production company(s)',
 'Production location(s)',
 'Productioncompanies ',
 'Productioncompany ',
 'Recorded',
 'Release date',
 'Released',
 'Revised Romanization',
 'Romanized',
 'Running time',
 'Russian',
 'Screen story by',
 'Screenplay by',
 'Simplifie

In [26]:
# review columns to understand the type of content in them.
wiki_movies_df[wiki_movies_df['title'] == 'The Insult']

,url,year,imdb_link,title,Directed by,Produced by,Screenplay by,Story by,Based on,Starring,...,Hepburn,Literally,Cantonese,Chinese,Yiddish,Arabic,Romanized,Russian,Hebrew,Polish
6834,https://en.wikipedia.org/wiki/The_Insult_(film),2018,https://www.imdb.com/title/tt7048622/,The Insult,Ziad Doueiri,"[Rachid Bouchareb, Jean Bréhat, Julie Gayet, A...",NaN,NaN,NaN,"[Adel Karam, Kamel El Basha]",...,NaN,Case No. 23,NaN,NaN,NaN,قضية رقم ٢٣,Qadiyya raqm 23,NaN,NaN,NaN


In [27]:
# review columns as a dictionary to understand the type of content in them.(easier to read)
wiki_movies_df.loc[wiki_movies_df['title'] == 'The Insult'].to_dict()

{'url': {6834: 'https://en.wikipedia.org/wiki/The_Insult_(film)'},
 'year': {6834: 2018},
 'imdb_link': {6834: 'https://www.imdb.com/title/tt7048622/'},
 'title': {6834: 'The Insult'},
 'Directed by': {6834: 'Ziad Doueiri'},
 'Produced by': {6834: ['Rachid Bouchareb',
   'Jean Bréhat',
   'Julie Gayet',
   'Antoun Sehnaoui',
   'Nadia Turincev']},
 'Screenplay by': {6834: nan},
 'Story by': {6834: nan},
 'Based on': {6834: nan},
 'Starring': {6834: ['Adel Karam', 'Kamel El Basha']},
 'Narrated by': {6834: nan},
 'Music by': {6834: 'Éric Neveux'},
 'Cinematography': {6834: 'Tommaso Fiorilli'},
 'Edited by': {6834: 'Dominique Marcombe'},
 'Productioncompany ': {6834: ['Ezekiel Films',
   'Tessalit Productions',
   'Rouge International']},
 'Distributed by': {6834: ['Diaphana Films',
   '(France)',
   'Italia Film',
   '(Lebanon']},
 'Release date': {6834: ['31 August 2017',
   '(',
   '2017-08-31',
   ')',
   '(',
   'Venice',
   ')',
   '14 September 2017',
   '(',
   '2017-09-14',
   '

In [28]:
insult_alt_titles = [wiki_movies_df[wiki_movies_df['title'] == 'The Insult']['Literally'], 
              wiki_movies_df[wiki_movies_df['title'] == 'The Insult']['Arabic'],
             wiki_movies_df[wiki_movies_df['title'] == 'The Insult']['Romanized']]
insult_alt_titles

[6834    Case No. 23
 Name: Literally, dtype: object,
 6834    قضية رقم ٢٣
 Name: Arabic, dtype: object,
 6834    Qadiyya raqm 23
 Name: Romanized, dtype: object]

In [29]:
# review columns as a dictionary to understand the type of content in them.(easier to read)
wiki_movies_df.loc[wiki_movies_df['title'] == 'Capernaum'].to_dict()

{'url': {7058: 'https://en.wikipedia.org/wiki/Capernaum_(film)'},
 'year': {7058: 2018},
 'imdb_link': {7058: 'https://www.imdb.com/title/tt8267604/'},
 'title': {7058: 'Capernaum'},
 'Directed by': {7058: 'Nadine Labaki'},
 'Produced by': {7058: ['Michel Merkt', 'Khaled Mouzanar']},
 'Screenplay by': {7058: ['Nadine Labaki',
   'Jihad Hojaily',
   'Michelle Keserwany']},
 'Story by': {7058: ['Georges Khabbaz',
   'Nadine Labaki',
   'Michelle Keserwany',
   'Jihad Hojaily',
   'Khaled Mouzanar']},
 'Based on': {7058: nan},
 'Starring': {7058: ['Zain Al Rafeea',
   'Yordanos Shiferaw',
   'Boluwatife Bankole',
   'Kawthar Al Haddad',
   'Fadi Kamel Youssef',
   'Nour el Husseini',
   'Alaa Chouchnieh',
   'Cedra Izam',
   'Nadine Labaki',
   'Joseph Jimbazian',
   'Farah Hasno']},
 'Narrated by': {7058: nan},
 'Music by': {7058: 'Khaled Mouzanar'},
 'Cinematography': {7058: 'Christopher Aoun'},
 'Edited by': {7058: 'Konstantin Bock'},
 'Productioncompany ': {7058: 'Mooz Films'},
 'Dist

In [30]:
Capernaum_alt_titles = [wiki_movies_df[wiki_movies_df['title'] == 'Capernaum']['Literally'], 
              wiki_movies_df[wiki_movies_df['title'] == 'Capernaum']['Arabic'],
             wiki_movies_df[wiki_movies_df['title'] == 'Capernaum']['Romanized']]
Capernaum_alt_titles

[7058    NaN
 Name: Literally, dtype: object,
 7058    کفرناحوم‎
 Name: Arabic, dtype: object,
 7058    NaN
 Name: Romanized, dtype: object]

In [56]:
# Create a function(1) to clean each entry
def clean_movie(movie):
    movie = dict(movie) # Create a non-destructive copy
    alt_titles = {} # Step 1. Make an empty dict to hold all of the alternative titles.
    
    # Combine alternate titles into one list.
    # Step 2. Loop through a list of all alternative title keys
    for key in ['Also known as', 'Arabic', 'Cantonese', 'Chinese', 'French',
                'Hangul', 'Hebrew', 'Hepburn', 'Japanese', 'Literally',
                'Mandarin', 'McCune–Reischauer','Original title', 'Polish',
                'Revised Romanization', 'Romanized', 'Russian',
                'Simplified', 'Traditional', 'Yiddish']:
        # Step 2a. Check if the current key exists in the movie object.
        if key in movie:
            # Step 2b. If so, remove the key-value pair and add to the alternative titles dictionary.
            alt_titles[key] = movie[key]
            movie.pop(key)
    # Step 3. After looping through every key, add the alternative titles dict to the movie object.
    if len(alt_titles) > 0:
        movie['alt_titles'] = alt_titles
        
    return movie

In [57]:
# Make a list of clean movies using list comprehension and the function(1).
clean_movies = [clean_movie(movie) for movie in wiki_movies]
len(clean_movies)

7076

In [ ]:
# Check the first 2 items in the movies list.
i = 0
while i < 2:
    print([i], wiki_movies[i])
    i += 1

In [ ]:
# Check the first 2 items in the movies list.
i = 0
while i < 2:
    print([i], clean_movies[i])
    i += 1

In [52]:
# Check new column 'alt_titles' in the movies list. notice the 'Arabic' key-value pair within alt title.
x = 7058
print(clean_movies[x])

{'url': 'https://en.wikipedia.org/wiki/Capernaum_(film)', 'year': 2018, 'imdb_link': 'https://www.imdb.com/title/tt8267604/', 'title': 'Capernaum', 'Directed by': 'Nadine Labaki', 'Produced by': ['Michel Merkt', 'Khaled Mouzanar'], 'Screenplay by': ['Nadine Labaki', 'Jihad Hojaily', 'Michelle Keserwany'], 'Story by': ['Georges Khabbaz', 'Nadine Labaki', 'Michelle Keserwany', 'Jihad Hojaily', 'Khaled Mouzanar'], 'Starring': ['Zain Al Rafeea', 'Yordanos Shiferaw', 'Boluwatife Bankole', 'Kawthar Al Haddad', 'Fadi Kamel Youssef', 'Nour el Husseini', 'Alaa Chouchnieh', 'Cedra Izam', 'Nadine Labaki', 'Joseph Jimbazian', 'Farah Hasno'], 'Music by': 'Khaled Mouzanar', 'Cinematography': 'Christopher Aoun', 'Edited by': 'Konstantin Bock', 'Productioncompany ': 'Mooz Films', 'Distributed by': 'Sony Pictures Classics', 'Release date': ['17 May 2018', '(', '2018-05-17', ')', '(', 'Cannes', ')', '20 September 2018', '(', '2018-09-20', ')', '(Lebanon)'], 'Running time': '126 minutes', 'Country': 'Leb

In [58]:
# Set wiki_movies_df to be the DataFrame created from clean_movies. notice new column 'alt_titles'.
wiki_movies_df = pd.DataFrame(clean_movies)
wiki_movies_df.head()

,url,year,imdb_link,title,Directed by,Produced by,Screenplay by,Story by,Based on,Starring,...,Released,Recorded,Venue,Length,Label,Director,Producer,Animation by,Color process,Screen story by
0,https://en.wikipedia.org/wiki/The_Adventures_o...,1990,https://www.imdb.com/title/tt0098987/,The Adventures of Ford Fairlane,Renny Harlin,"[Steve Perry, Joel Silver]","[David Arnott, James Cappe, Daniel Waters]","[David Arnott, James Cappe]","[Characters, by Rex Weiner]","[Andrew Dice Clay, Wayne Newton, Priscilla Pre...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"https://en.wikipedia.org/wiki/After_Dark,_My_S...",1990,https://www.imdb.com/title/tt0098994/,"After Dark, My Sweet",James Foley,"[Ric Kidney, Robert Redlin]","[James Foley, Robert Redlin]",NaN,"[the novel, After Dark, My Sweet, by, Jim Thom...","[Jason Patric, Rachel Ward, Bruce Dern, George...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://en.wikipedia.org/wiki/Air_America_(film),1990,https://www.imdb.com/title/tt0099005/,Air America,Roger Spottiswoode,Daniel Melnick,"[John Eskow, Richard Rush]",NaN,"[Air America, by, Christopher Robbins]","[Mel Gibson, Robert Downey Jr., Nancy Travis, ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://en.wikipedia.org/wiki/Alice_(1990_film),1990,https://www.imdb.com/title/tt0099012/,Alice,Woody Allen,Robert Greenhut,NaN,NaN,NaN,"[Alec Baldwin, Blythe Danner, Judy Davis, Mia ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://en.wikipedia.org/wiki/Almost_an_Angel,1990,https://www.imdb.com/title/tt0099018/,Almost an Angel,John Cornell,John Cornell,NaN,NaN,NaN,"[Paul Hogan, Elias Koteas, Linda Kozlowski]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [59]:
sorted(wiki_movies_df.columns.tolist())

['Adaptation by',
 'Animation by',
 'Audio format',
 'Based on',
 'Box office',
 'Budget',
 'Cinematography',
 'Color process',
 'Composer(s)',
 'Country',
 'Country of origin',
 'Created by',
 'Directed by',
 'Director',
 'Distributed by',
 'Distributor',
 'Edited by',
 'Editor(s)',
 'Executive producer(s)',
 'Followed by',
 'Genre',
 'Label',
 'Language',
 'Length',
 'Music by',
 'Narrated by',
 'Original language(s)',
 'Original network',
 'Original release',
 'Picture format',
 'Preceded by',
 'Produced by',
 'Producer',
 'Producer(s)',
 'Production company(s)',
 'Production location(s)',
 'Productioncompanies ',
 'Productioncompany ',
 'Recorded',
 'Release date',
 'Released',
 'Running time',
 'Screen story by',
 'Screenplay by',
 'Starring',
 'Story by',
 'Suggested by',
 'Theme music composer',
 'Venue',
 'Voices of',
 'Written by',
 'alt_titles',
 'imdb_link',
 'title',
 'url',
 'year']

In [54]:
col = wiki_movies_df['alt_titles'].tolist()
print(col)

[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,

In [65]:
# Re-Create and expand function(2) to clean each entry.
def clean_movie(movie):
    movie = dict(movie) # Create a non-destructive copy
    alt_titles = {} # Step 1. Make an empty dict to hold all of the alternative titles.
    
    # Combine alternate titles into one list.
    # Step 2. Loop through a list of all alternative title keys
    for key in ['Also known as', 'Arabic', 'Cantonese', 'Chinese', 'French', 
                'Hangul', 'Hebrew', 'Hepburn', 'Japanese', 'Literally', 
                'Mandarin', 'McCune–Reischauer','Original title', 'Polish', 
                'Revised Romanization', 'Romanized', 'Russian', 
                'Simplified', 'Traditional', 'Yiddish']:
        # Step 2a. Check if the current key exists in the movie object.
        if key in movie:
            # Step 2b. If so, remove the key-value pair and add to the alternative titles dictionary.
            alt_titles[key] = movie[key]
            movie.pop(key)
    # Step 3. After looping through every key, add the alternative titles dict to the movie object.
    if len(alt_titles) > 0:
        movie['alt_titles'] = alt_titles
    
    # merge column names
    def change_column_name(old_name, new_name):
        if old_name in movie:
            movie[new_name] = movie.pop(old_name)
    change_column_name('Adaptation by', 'Writer(s)')
    change_column_name('Country of origin', 'Country')
    change_column_name('Directed by', 'Director')
    change_column_name('Distributed by', 'Distributor')
    change_column_name('Edited by', 'Editor(s)')
    change_column_name('Length', 'Running time')
    change_column_name('Original release', 'Release date')
    change_column_name('Music by', 'Composer(s)')
    change_column_name('Produced by', 'Producer(s)')
    change_column_name('Producer', 'Producer(s)')
    change_column_name('Productioncompanies ', 'Production company(s)')
    change_column_name('Productioncompany ', 'Production company(s)')
    change_column_name('Released', 'Release Date')
    change_column_name('Release Date', 'Release date')
    change_column_name('Screen story by', 'Writer(s)')
    change_column_name('Screenplay by', 'Writer(s)')
    change_column_name('Story by', 'Writer(s)')
    change_column_name('Theme music composer', 'Composer(s)')
    change_column_name('Written by', 'Writer(s)')
        
    return movie

In [66]:
# rerun list comprehension to clean wiki_movies and recreate wiki_movies_df.
clean_movies = [clean_movie(movie) for movie in wiki_movies]
len(clean_movies)

7076

In [67]:
# Recreat the wiki DataFrame.
wiki_movies_df = pd.DataFrame(clean_movies)
wiki_movies_df.head()

,url,year,imdb_link,title,Based on,Starring,Narrated by,Cinematography,Release date,Running time,...,Followed by,Created by,Preceded by,Suggested by,alt_titles,Recorded,Venue,Label,Animation by,Color process
0,https://en.wikipedia.org/wiki/The_Adventures_o...,1990,https://www.imdb.com/title/tt0098987/,The Adventures of Ford Fairlane,"[Characters, by Rex Weiner]","[Andrew Dice Clay, Wayne Newton, Priscilla Pre...","Andrew ""Dice"" Clay",Oliver Wood,"[July 11, 1990, (, 1990-07-11, )]",102 minutes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"https://en.wikipedia.org/wiki/After_Dark,_My_S...",1990,https://www.imdb.com/title/tt0098994/,"After Dark, My Sweet","[the novel, After Dark, My Sweet, by, Jim Thom...","[Jason Patric, Rachel Ward, Bruce Dern, George...",NaN,Mark Plummer,"[May 17, 1990, (, 1990-05-17, ), (Cannes Film ...",114 minutes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://en.wikipedia.org/wiki/Air_America_(film),1990,https://www.imdb.com/title/tt0099005/,Air America,"[Air America, by, Christopher Robbins]","[Mel Gibson, Robert Downey Jr., Nancy Travis, ...",NaN,Roger Deakins,"[August 10, 1990, (, 1990-08-10, )]",113 minutes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://en.wikipedia.org/wiki/Alice_(1990_film),1990,https://www.imdb.com/title/tt0099012/,Alice,NaN,"[Alec Baldwin, Blythe Danner, Judy Davis, Mia ...",NaN,Carlo Di Palma,"[December 25, 1990, (, 1990-12-25, )]",106 minutes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://en.wikipedia.org/wiki/Almost_an_Angel,1990,https://www.imdb.com/title/tt0099018/,Almost an Angel,NaN,"[Paul Hogan, Elias Koteas, Linda Kozlowski]",NaN,Russell Boyd,"December 19, 1990",95 minutes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [68]:
# Get the final (39) columns in the clean df.
sorted(wiki_movies_df.columns.tolist())

['Animation by',
 'Audio format',
 'Based on',
 'Box office',
 'Budget',
 'Cinematography',
 'Color process',
 'Composer(s)',
 'Country',
 'Created by',
 'Director',
 'Distributor',
 'Editor(s)',
 'Executive producer(s)',
 'Followed by',
 'Genre',
 'Label',
 'Language',
 'Narrated by',
 'Original language(s)',
 'Original network',
 'Picture format',
 'Preceded by',
 'Producer(s)',
 'Production company(s)',
 'Production location(s)',
 'Recorded',
 'Release date',
 'Running time',
 'Starring',
 'Suggested by',
 'Venue',
 'Voices of',
 'Writer(s)',
 'alt_titles',
 'imdb_link',
 'title',
 'url',
 'year']

## Remove Duplicate Rows using Regex

In [71]:
# Use Pandas built-in string method to extract IMDb IDs from imdb_link and put them in a new column.
wiki_movies_df['imdb_id'] = wiki_movies_df['imdb_link'].str.extract(r'(tt\d{7})')
print(len(wiki_movies_df))
print(len(wiki_movies_df['imdb_id']))

7076
7076


In [73]:
# Drop duplicates of IMDb IDs using drop_duplicates() and only consider IMDb ID by using subset argument, and inplace to it's performed on the selected DF.
wiki_movies_df.drop_duplicates(subset='imdb_id', inplace=True)
# Show how many rows were dropped.
print(len(wiki_movies_df))

7033


In [74]:
# Check DF with one added column (imdb_id) at the end. (40 columns)
wiki_movies_df.head()

,url,year,imdb_link,title,Based on,Starring,Narrated by,Cinematography,Release date,Running time,...,Created by,Preceded by,Suggested by,alt_titles,Recorded,Venue,Label,Animation by,Color process,imdb_id
0,https://en.wikipedia.org/wiki/The_Adventures_o...,1990,https://www.imdb.com/title/tt0098987/,The Adventures of Ford Fairlane,"[Characters, by Rex Weiner]","[Andrew Dice Clay, Wayne Newton, Priscilla Pre...","Andrew ""Dice"" Clay",Oliver Wood,"[July 11, 1990, (, 1990-07-11, )]",102 minutes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tt0098987
1,"https://en.wikipedia.org/wiki/After_Dark,_My_S...",1990,https://www.imdb.com/title/tt0098994/,"After Dark, My Sweet","[the novel, After Dark, My Sweet, by, Jim Thom...","[Jason Patric, Rachel Ward, Bruce Dern, George...",NaN,Mark Plummer,"[May 17, 1990, (, 1990-05-17, ), (Cannes Film ...",114 minutes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tt0098994
2,https://en.wikipedia.org/wiki/Air_America_(film),1990,https://www.imdb.com/title/tt0099005/,Air America,"[Air America, by, Christopher Robbins]","[Mel Gibson, Robert Downey Jr., Nancy Travis, ...",NaN,Roger Deakins,"[August 10, 1990, (, 1990-08-10, )]",113 minutes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tt0099005
3,https://en.wikipedia.org/wiki/Alice_(1990_film),1990,https://www.imdb.com/title/tt0099012/,Alice,NaN,"[Alec Baldwin, Blythe Danner, Judy Davis, Mia ...",NaN,Carlo Di Palma,"[December 25, 1990, (, 1990-12-25, )]",106 minutes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tt0099012
4,https://en.wikipedia.org/wiki/Almost_an_Angel,1990,https://www.imdb.com/title/tt0099018/,Almost an Angel,NaN,"[Paul Hogan, Elias Koteas, Linda Kozlowski]",NaN,Russell Boyd,"December 19, 1990",95 minutes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tt0099018


## Remove Mostly Null Columns

In [77]:
# Check how many null values are in each column out of 40 cols for 7,033 rows.
wiki_movies_df.isnull().sum()

url                          0
year                         0
imdb_link                    0
title                        1
Based on                  4852
Starring                   184
Narrated by               6752
Cinematography             691
Release date                32
Running time               139
Country                    236
Language                   244
Budget                    2295
Box office                1548
Director                     0
Distributor                357
Editor(s)                  548
Composer(s)                518
Producer(s)                202
Production company(s)     1678
Writer(s)                  199
Genre                     6923
Original language(s)      6875
Original network          6908
Executive producer(s)     6936
Production location(s)    6986
Picture format            6969
Audio format              6972
Voices of                 7031
Followed by               7024
Created by                7023
Preceded by               7023
Suggeste

In [79]:
# another way using list comprehension: Check how many null values are in each column out of 40 cols for 7,033 rows.
[[column, wiki_movies_df[column].isnull().sum()] for column in wiki_movies_df.columns]

[['url', 0],
 ['year', 0],
 ['imdb_link', 0],
 ['title', 1],
 ['Based on', 4852],
 ['Starring', 184],
 ['Narrated by', 6752],
 ['Cinematography', 691],
 ['Release date', 32],
 ['Running time', 139],
 ['Country', 236],
 ['Language', 244],
 ['Budget', 2295],
 ['Box office', 1548],
 ['Director', 0],
 ['Distributor', 357],
 ['Editor(s)', 548],
 ['Composer(s)', 518],
 ['Producer(s)', 202],
 ['Production company(s)', 1678],
 ['Writer(s)', 199],
 ['Genre', 6923],
 ['Original language(s)', 6875],
 ['Original network', 6908],
 ['Executive producer(s)', 6936],
 ['Production location(s)', 6986],
 ['Picture format', 6969],
 ['Audio format', 6972],
 ['Voices of', 7031],
 ['Followed by', 7024],
 ['Created by', 7023],
 ['Preceded by', 7023],
 ['Suggested by', 7032],
 ['alt_titles', 7012],
 ['Recorded', 7031],
 ['Venue', 7032],
 ['Label', 7031],
 ['Animation by', 7031],
 ['Color process', 7032],
 ['imdb_id', 0]]

In [80]:
# a third way is using 'for' loop and 'print' statement to find the null sum in each column.
for column in (wiki_movies_df.columns):
    print([column, wiki_movies_df[column].isnull().sum()])

['url', 0]
['year', 0]
['imdb_link', 0]
['title', 1]
['Based on', 4852]
['Starring', 184]
['Narrated by', 6752]
['Cinematography', 691]
['Release date', 32]
['Running time', 139]
['Country', 236]
['Language', 244]
['Budget', 2295]
['Box office', 1548]
['Director', 0]
['Distributor', 357]
['Editor(s)', 548]
['Composer(s)', 518]
['Producer(s)', 202]
['Production company(s)', 1678]
['Writer(s)', 199]
['Genre', 6923]
['Original language(s)', 6875]
['Original network', 6908]
['Executive producer(s)', 6936]
['Production location(s)', 6986]
['Picture format', 6969]
['Audio format', 6972]
['Voices of', 7031]
['Followed by', 7024]
['Created by', 7023]
['Preceded by', 7023]
['Suggested by', 7032]
['alt_titles', 7012]
['Recorded', 7031]
['Venue', 7032]
['Label', 7031]
['Animation by', 7031]
['Color process', 7032]
['imdb_id', 0]


In [83]:
# Plan to remove the nulls by first making a list of columns with <90% nulls, by tweaking the list comprehension statement. 
wiki_columns_to_keep = [column for column in wiki_movies_df.columns if wiki_movies_df[column].isnull().sum() < len(wiki_movies_df) * 0.9]
len(wiki_columns_to_keep)

21

In [84]:
type(wiki_columns_to_keep)

list

In [86]:
# Recreate the DF with only the columns to keep that have > 10% not null (i.e. <90% null).
wiki_movies_df = wiki_movies_df[wiki_columns_to_keep]
wiki_movies_df.head()

,url,year,imdb_link,title,Based on,Starring,Cinematography,Release date,Running time,Country,...,Budget,Box office,Director,Distributor,Editor(s),Composer(s),Producer(s),Production company(s),Writer(s),imdb_id
0,https://en.wikipedia.org/wiki/The_Adventures_o...,1990,https://www.imdb.com/title/tt0098987/,The Adventures of Ford Fairlane,"[Characters, by Rex Weiner]","[Andrew Dice Clay, Wayne Newton, Priscilla Pre...",Oliver Wood,"[July 11, 1990, (, 1990-07-11, )]",102 minutes,United States,...,$20 million,$21.4 million,Renny Harlin,20th Century Fox,Michael Tronick,"[Cliff Eidelman, Yello]","[Steve Perry, Joel Silver]",Silver Pictures,"[David Arnott, James Cappe]",tt0098987
1,"https://en.wikipedia.org/wiki/After_Dark,_My_S...",1990,https://www.imdb.com/title/tt0098994/,"After Dark, My Sweet","[the novel, After Dark, My Sweet, by, Jim Thom...","[Jason Patric, Rachel Ward, Bruce Dern, George...",Mark Plummer,"[May 17, 1990, (, 1990-05-17, ), (Cannes Film ...",114 minutes,United States,...,$6 million,$2.7 million,James Foley,Avenue Pictures,Howard E. Smith,Maurice Jarre,"[Ric Kidney, Robert Redlin]",Avenue Pictures,"[James Foley, Robert Redlin]",tt0098994
2,https://en.wikipedia.org/wiki/Air_America_(film),1990,https://www.imdb.com/title/tt0099005/,Air America,"[Air America, by, Christopher Robbins]","[Mel Gibson, Robert Downey Jr., Nancy Travis, ...",Roger Deakins,"[August 10, 1990, (, 1990-08-10, )]",113 minutes,United States,...,$35 million,"$57,718,089",Roger Spottiswoode,TriStar Pictures,"[John Bloom, Lois Freeman-Fox]",Charles Gross,Daniel Melnick,"[Carolco Pictures, IndieProd Company]","[John Eskow, Richard Rush]",tt0099005
3,https://en.wikipedia.org/wiki/Alice_(1990_film),1990,https://www.imdb.com/title/tt0099012/,Alice,NaN,"[Alec Baldwin, Blythe Danner, Judy Davis, Mia ...",Carlo Di Palma,"[December 25, 1990, (, 1990-12-25, )]",106 minutes,United States,...,$12 million,"$7,331,647",Woody Allen,Orion Pictures,Susan E. Morse,NaN,Robert Greenhut,NaN,Woody Allen,tt0099012
4,https://en.wikipedia.org/wiki/Almost_an_Angel,1990,https://www.imdb.com/title/tt0099018/,Almost an Angel,NaN,"[Paul Hogan, Elias Koteas, Linda Kozlowski]",Russell Boyd,"December 19, 1990",95 minutes,US,...,$25 million,"$6,939,946 (USA)",John Cornell,Paramount Pictures,David Stiven,Maurice Jarre,John Cornell,NaN,Paul Hogan,tt0099018


In [87]:
# Notice the df is down to (21) columns and the 'alt_title' column is gone. 
wiki_movies_df.columns.tolist()

['url',
 'year',
 'imdb_link',
 'title',
 'Based on',
 'Starring',
 'Cinematography',
 'Release date',
 'Running time',
 'Country',
 'Language',
 'Budget',
 'Box office',
 'Director',
 'Distributor',
 'Editor(s)',
 'Composer(s)',
 'Producer(s)',
 'Production company(s)',
 'Writer(s)',
 'imdb_id']

In [92]:
# look for the alt_titles column in the arabic movie 'The Insult', now with a different index. 
wiki_movies_df.loc[wiki_movies_df['title'] == 'The Insult'].to_dict()
# it's gone!

{'url': {6834: 'https://en.wikipedia.org/wiki/The_Insult_(film)'},
 'year': {6834: 2018},
 'imdb_link': {6834: 'https://www.imdb.com/title/tt7048622/'},
 'title': {6834: 'The Insult'},
 'Based on': {6834: nan},
 'Starring': {6834: ['Adel Karam', 'Kamel El Basha']},
 'Cinematography': {6834: 'Tommaso Fiorilli'},
 'Release date': {6834: ['31 August 2017',
   '(',
   '2017-08-31',
   ')',
   '(',
   'Venice',
   ')',
   '14 September 2017',
   '(',
   '2017-09-14',
   ')',
   '(Lebanon)',
   '31 January 2018',
   '(',
   '2018-01-31',
   ')',
   '(France)']},
 'Running time': {6834: '112 minutes'},
 'Country': {6834: ['France', 'Lebanon']},
 'Language': {6834: 'Lebanese Arabic'},
 'Budget': {6834: nan},
 'Box office': {6834: '$1.6 million'},
 'Director': {6834: 'Ziad Doueiri'},
 'Distributor': {6834: ['Diaphana Films',
   '(France)',
   'Italia Film',
   '(Lebanon']},
 'Editor(s)': {6834: 'Dominique Marcombe'},
 'Composer(s)': {6834: 'Éric Neveux'},
 'Producer(s)': {6834: ['Rachid Bouchar

## Convert and Parse the Wiki Data

In [97]:
# Check the data types of columns in wiki dataframe.
wiki_movies_df.dtypes

url                      object
year                      int64
imdb_link                object
title                    object
Based on                 object
Starring                 object
Cinematography           object
Release date             object
Running time             object
Country                  object
Language                 object
Budget                   object
Box office               object
Director                 object
Distributor              object
Editor(s)                object
Composer(s)              object
Producer(s)              object
Production company(s)    object
Writer(s)                object
imdb_id                  object
dtype: object

In [98]:
len(wiki_movies_df)

7033

### Parse the Box Office Data

In [ ]:
# Collect the rows with box office data. it will be defined as a series. 
box_office = wiki_movies_df['Box office'].dropna()
type(box_office)

In [181]:
# Check the number of data points that exists in the box office series. 
len(box_office)

5485

In [103]:
print(box_office[0:5])

0       $21.4 million
1        $2.7 million
2         $57,718,089
3          $7,331,647
4    $6,939,946 (USA)
Name: Box office, dtype: object


In [108]:
print(box_office.sample(n=5))

3949                            $12 million
3070                            $51,801,187
7042                          $35.2 million
4656                           $128 million
1761    [$2,821,671, [, citation needed, ]]
Name: Box office, dtype: object


In [110]:
# Regex only work on strings, so make sure all box office data is string. 
# 1. define a function to find the type of data in the box office series.
def is_not_a_string(x):
    return type(x) != str
# 2. use the map() function to see the values that are not strings. 
box_office[box_office.map(is_not_a_string)]

34                           [US$, 4,212,828]
54      [$6,698,361 (, United States, ), [2]]
74                    [$6,488,144, (US), [1]]
126                [US$1,531,489, (domestic)]
130                          [US$, 4,803,039]
                        ...                  
6980               [$99.6, million, [4], [5]]
6994                   [$365.6, million, [1]]
6995                         [$53.8, million]
7015                     [$435, million, [7]]
7048                   [$529.3, million, [4]]
Name: Box office, Length: 135, dtype: object

In [111]:
# another way is to use anonymous lambda function inside map() in one code line. 
box_office[box_office.map(lambda x: type(x) != str)]

34                           [US$, 4,212,828]
54      [$6,698,361 (, United States, ), [2]]
74                    [$6,488,144, (US), [1]]
126                [US$1,531,489, (domestic)]
130                          [US$, 4,803,039]
                        ...                  
6980               [$99.6, million, [4], [5]]
6994                   [$365.6, million, [1]]
6995                         [$53.8, million]
7015                     [$435, million, [7]]
7048                   [$529.3, million, [4]]
Name: Box office, Length: 135, dtype: object

In [129]:
# some box office data points are stored as lists, 
# use simple space as the joining character and apply join() function only when data points are lists.
box_office = box_office.apply(lambda x: ' '.join(x) if type(x) == list else x)
box_office.sample(n=5)

2297                                         $5.5 million
3132                                        $38.3 million
56                                      $2.7 million (US)
5693                                             $408,015
4641    US$22,211,426 (worldwide) £1,034,659 [3] (Unit...
Name: Box office, dtype: object

In [132]:
# Create the second form to match the box office data in form "$123.4 million or billion".
form_one = r"\$\d+\.?\d*\s*[mb]illion"

In [134]:
# Count up how many box office values match the first form, ignore upper- lowercase, parse the non-string data to False.
box_office.str.contains(form_one, flags=re.IGNORECASE, na=False).sum()

3896

In [135]:
# Create the first form to match the box office data in form "$123,456,789"
form_two = r"\$\d{1,3}(?:,\d{3})+"

In [136]:
# Count up how many box office values match the second form, ignore upper- lowercase, parse the non-string data to False.
box_office.str.contains(form_two, flags=re.IGNORECASE, na=False).sum()

1544

In [137]:
# Most of the box office values are described by either form (out of 5,485) movies.
3896+1544

5440

In [140]:
# Create two boolean series to look for matches then use to find the values that don't match either.
matches_form_one = box_office.str.contains(form_one, flags=re.IGNORECASE, na=False)
matches_form_two = box_office.str.contains(form_two, flags=re.IGNORECASE, na=False)

In [141]:
# Select the box office values that don't match either forms.
box_office[(not matches_form_one) and (not matches_form_two)]

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [144]:
# Select the box office values that don't match either forms, using Pandas element-wise logical operators.
# tilde ~ (similar to 'not'), ampersand & is 'and', pipe | is 'or'.
box_office[~matches_form_one & ~matches_form_two]

34                         US$ 4,212,828
79                              $335.000
110                   $4.35-4.37 million
130                        US$ 4,803,039
600                           $5000 (US)
731                         $ 11,146,270
957                             $ 50,004
1070                          35,254,617
1147    $ 407,618 (U.S.) (sub-total) [1]
1446                        $ 11,829,959
1480                          £3 million
1611                            $520.000
1865                        ¥1.1 billion
2032                                 N/A
2091                                $309
2130               US$ 171.8 million [9]
2257                   US$ 3,395,581 [1]
2263            $ 1,223,034 ( domestic )
2347                            $282.175
2638            $ 104,883 (US sub-total)
2665         926,423 admissions (France)
2697      $ 1.7 million (US) (sub-total)
2823                            $414.000
2924                            $621.000
3088           $

In [152]:
# 1. Fix pattern match where there's a space in between the dollar sign and the number. 
form_one = r"\$\s*\d+\.?\d*\s*[mb]illion"
form_two = r"\$\s*\d{1,3}(?:,\d{3})+"

In [153]:
# 2. Fix pattern match where there's a period as a thousand's separator, not a comma, but don't capture any values like 1.234 billion.
form_two = r"\$\s*\d{1,3}(?:[,\.]\d{3})+(?!\s[mb]illion)"

In [154]:
# 3. Fix pattern match where the values are given as a range by replacing the hyphen after $number with only a $.
box_office = box_office.replace(r"\$.*[-—–](?![a-z])", '$', regex=True)


In [158]:
# 4. Fix pattern match where'Million' is sometimes misspelled as 'millon'.
form_one = r"\$\s*\d+\.?\d*\s*[mb]illi?on"

In [157]:
box_office[~matches_form_one & ~matches_form_two]

34                         US$ 4,212,828
79                              $335.000
110                        $4.37 million
130                        US$ 4,803,039
600                           $5000 (US)
731                         $ 11,146,270
957                             $ 50,004
1070                          35,254,617
1147    $ 407,618 (U.S.) (sub-total) [1]
1446                        $ 11,829,959
1480                          £3 million
1611                            $520.000
1865                        ¥1.1 billion
2032                                 N/A
2091                                $309
2130               US$ 171.8 million [9]
2257                   US$ 3,395,581 [1]
2263            $ 1,223,034 ( domestic )
2347                            $282.175
2638            $ 104,883 (US sub-total)
2665         926,423 admissions (France)
2697      $ 1.7 million (US) (sub-total)
2823                            $414.000
2924                            $621.000
3088            

In [163]:
# Check our work by creating two boolean series to look for matches then use to find the values that don't match either.
match_form_one = box_office.str.contains(form_one, flags=re.IGNORECASE, na=False)
match_form_two = box_office.str.contains(form_two, flags=re.IGNORECASE, na=False)
box_office[~match_form_one & ~match_form_two]
# ignore the rest for they constitute a small percentage of the data, only (19) entries. 

600                      $5000 (US)
1070                     35,254,617
1480                     £3 million
1865                   ¥1.1 billion
2032                            N/A
2091                           $309
2665    926,423 admissions (France)
3631                            TBA
3879       CN¥3.650 million (China)
4116                     £7,385,434
4306                            $30
4561        $45.2k (only in Turkey)
5447                          £2.56
5784                       413 733$
6013                        Unknown
6369                          $111k
6370                           $588
6593                 less than $372
6843                        8 crore
Name: Box office, dtype: object

In [165]:
# Extract the box office data that matches either forms using regular expression, and convert to a DataFrame. 
box_office.str.extract(f'({form_one}|{form_two})')

,0
0,$21.4 million
1,$2.7 million
2,"$57,718,089"
3,"$7,331,647"
4,"$6,939,946"
...,...
7070,$19.4 million
7071,$41.9 million
7072,$76.1 million
7073,$38.4 million


In [170]:
# Define a function to turn the extracted values (strings) to numberic (floating-point decimal) values. 
# Since we're working directly with strings, we'll use the re module to access the regex functions. 
# Use re.match(pattern, string) to see if our string matches a pattern. 
# Use re.sub(pattern, replacement_string, string) to remove dollar signs, spaces, commas, and letter, if necessary.
# Convert all strings to floats, multiply by the right amount, and return the value.

def parse_dollars(s):
    # if s is not a string, return NaN
    if type(s) != str:
        return np.nan
    
    # if input is of the form $###.# million
    if re.match(r'\$\s*\d+\.?\d*\s*milli?on', s, flags=re.IGNORECASE):
        
        # remove dollar sign and "million"
        s = re.sub('\$|\s|[a-zA-Z]','', s)
        
        # convert to float and multiply by a million
        value = float(s) * 10**6
        
        # return value
        return value
        
    # if input is of the form $###.# billion 
    elif re.match(r'\$\s*\d+\.?\d*\s*billi?on', s, flags=re.IGNORECASE):
    
        # remove dollar sign and "billion"
        s = re.sub('\$|\s|[a-zA-Z]','', s)
        
        # convert to float and multiply by a billion
        value = float(s) * 10**9
        
        # return value
        return value
        
    # if input is of the form $###,###,###
    elif re.match(r'\$\s*\d{1,3}(?:[,\.]\d{3})+(?!\s[mb]illion)', s, flags=re.IGNORECASE):
        
        # remove dollar sign and commas
        s = re.sub('\$|,','', s)
        
        # convert to float
        value = float(s)
        
        # return value
        return value
    
    # otherwise, return NaN
    else: 
        return np.nan
    

In [171]:
# now we have all we need to parse the box office values to numeric values. 
# extract the values from box_office using str.extract. then apply the parse_dollars to the first col of DF returned by str.extract.
wiki_movies_df['box_office'] = box_office.str.extract(f'({form_one}|{form_two})', flags=re.IGNORECASE)[0].apply(parse_dollars)

In [172]:
# Check the new column in the DF.
wiki_movies_df['box_office']

0       21400000.0
1        2700000.0
2       57718089.0
3        7331647.0
4        6939946.0
           ...    
7071    41900000.0
7072    76100000.0
7073    38400000.0
7074     5500000.0
7075           NaN
Name: box_office, Length: 7033, dtype: float64

In [ ]:
# Drop the 'Box office' column since it's no longer needed. 
wiki_movies_df.drop('Box office', axis=1, inplace=True)

In [176]:
wiki_movies_df.head()

,url,year,imdb_link,title,Based on,Starring,Cinematography,Release date,Running time,Country,...,Budget,Director,Distributor,Editor(s),Composer(s),Producer(s),Production company(s),Writer(s),imdb_id,box_office
0,https://en.wikipedia.org/wiki/The_Adventures_o...,1990,https://www.imdb.com/title/tt0098987/,The Adventures of Ford Fairlane,"[Characters, by Rex Weiner]","[Andrew Dice Clay, Wayne Newton, Priscilla Pre...",Oliver Wood,"[July 11, 1990, (, 1990-07-11, )]",102 minutes,United States,...,$20 million,Renny Harlin,20th Century Fox,Michael Tronick,"[Cliff Eidelman, Yello]","[Steve Perry, Joel Silver]",Silver Pictures,"[David Arnott, James Cappe]",tt0098987,21400000.0
1,"https://en.wikipedia.org/wiki/After_Dark,_My_S...",1990,https://www.imdb.com/title/tt0098994/,"After Dark, My Sweet","[the novel, After Dark, My Sweet, by, Jim Thom...","[Jason Patric, Rachel Ward, Bruce Dern, George...",Mark Plummer,"[May 17, 1990, (, 1990-05-17, ), (Cannes Film ...",114 minutes,United States,...,$6 million,James Foley,Avenue Pictures,Howard E. Smith,Maurice Jarre,"[Ric Kidney, Robert Redlin]",Avenue Pictures,"[James Foley, Robert Redlin]",tt0098994,2700000.0
2,https://en.wikipedia.org/wiki/Air_America_(film),1990,https://www.imdb.com/title/tt0099005/,Air America,"[Air America, by, Christopher Robbins]","[Mel Gibson, Robert Downey Jr., Nancy Travis, ...",Roger Deakins,"[August 10, 1990, (, 1990-08-10, )]",113 minutes,United States,...,$35 million,Roger Spottiswoode,TriStar Pictures,"[John Bloom, Lois Freeman-Fox]",Charles Gross,Daniel Melnick,"[Carolco Pictures, IndieProd Company]","[John Eskow, Richard Rush]",tt0099005,57718089.0
3,https://en.wikipedia.org/wiki/Alice_(1990_film),1990,https://www.imdb.com/title/tt0099012/,Alice,NaN,"[Alec Baldwin, Blythe Danner, Judy Davis, Mia ...",Carlo Di Palma,"[December 25, 1990, (, 1990-12-25, )]",106 minutes,United States,...,$12 million,Woody Allen,Orion Pictures,Susan E. Morse,NaN,Robert Greenhut,NaN,Woody Allen,tt0099012,7331647.0
4,https://en.wikipedia.org/wiki/Almost_an_Angel,1990,https://www.imdb.com/title/tt0099018/,Almost an Angel,NaN,"[Paul Hogan, Elias Koteas, Linda Kozlowski]",Russell Boyd,"December 19, 1990",95 minutes,US,...,$25 million,John Cornell,Paramount Pictures,David Stiven,Maurice Jarre,John Cornell,NaN,Paul Hogan,tt0099018,6939946.0


In [178]:
sorted(wiki_movies_df.columns.tolist())

['Based on',
 'Budget',
 'Cinematography',
 'Composer(s)',
 'Country',
 'Director',
 'Distributor',
 'Editor(s)',
 'Language',
 'Producer(s)',
 'Production company(s)',
 'Release date',
 'Running time',
 'Starring',
 'Writer(s)',
 'box_office',
 'imdb_id',
 'imdb_link',
 'title',
 'url',
 'year']

### Parse Budget Data

In [198]:
# Collect the rows with budget data. it will be defined as a series. 
budget = wiki_movies_df['Budget'].dropna()
type(budget)

pandas.core.series.Series

In [238]:
# Check the number of data points that exists in the budget series. 
len(budget)

4738

In [239]:
print(budget[0:5])

0    $20 million
1     $6 million
2    $35 million
3    $12 million
4    $25 million
Name: Budget, dtype: object


In [201]:
print(budget.sample(n=5))

3513             $40 million
4777             $22 million
1842             $32 million
6251    [$6.5, million, [2]]
6891             $21 million
Name: Budget, dtype: object


In [202]:
print(budget.sample(n=10))

5047         $17 million
4859       US$30 million
3000         $17 million
999          $60 million
6162         $40 million
4296         $80 million
5964         $60 million
4554    $70-88.5 million
6197         $13 million
5793            $250,000
Name: Budget, dtype: object


In [203]:
print(budget.sample(n=10))

6847                            $200 million
740                              $12 million
5777                              $8 million
6222    [$45 million, [, citation needed, ]]
1736                                $200,000
1226                                 Unknown
4695                             $62 million
3433                             $17 million
1292                             $27 million
3402                             $85 million
Name: Budget, dtype: object


In [204]:
# after sampling the 'budget' series, we see $, lists, million, commas, other alphabetical characters. 

In [205]:
# Regex only work on strings, so make sure all box office data is string. 
# Use the map() function to see the values that are not strings. 
# and use anonymous lambda function inside map() in one code line, to find the non-string entries. 
budget[budget.map(lambda x: type(x) != str)]

16                    [$2,500,000, [, citation needed, ]]
100                       [£960,000, [2], ($1.5 million)]
135                                     [USD$, 9 million]
142      [$23 million (estimated), [, citation needed, ]]
151                  [$14 million, [, citation needed, ]]
                              ...                        
6994                                  [$22, million, [1]]
6995                                     [$22.8, million]
7015                                  [$36, million, [6]]
7018    [$70 million (gross), [3], [4], $59 million (n...
7048                                 [$175, million, [4]]
Name: Budget, Length: 184, dtype: object

In [206]:
# Convert any lists to strings. 
# use simple space as the joining character and apply join() function only when data points are lists.
budget = budget.map(lambda x: ' '.join(x) if type(x) == list else x)

In [219]:
# Check the result of joining lists. 
print(budget[16])
print(budget[135])
print(budget[100])

$2,500,000 [ citation needed ]
USD$ 9 million
£960,000 [2] ($1.5 million)


In [220]:
# Remove any values between a dollar sign and a hyphen for budgets given in ranges. 
budget = budget.str.replace(r'\$.*[-—–](?![a-z])', '$', regex=True)

In [221]:
# Check the result of removing hyphens.
print(budget[4554])

$88.5 million


In [224]:
# Create the first form to match the budget data in form "$123.4 million or billion".
form_one = r"\$\s*\d+\.?\d*\s*[mb]illi?on"

In [225]:
# Count up how many budget values match the first form, ignore upper- lowercase, parse the non-string data to False.
budget.str.contains(form_one, flags=re.IGNORECASE, na=False).sum()

4363

In [226]:
# Create the first form to match the budget data in form "$123,456,789"
form_two = r"\$\s*\d{1,3}(?:[,\.]\d{3})+(?!\s[mb]illion)"

In [227]:
# Count up how many budget values match the second form, ignore upper- lowercase, parse the non-string data to False.
budget.str.contains(form_two, flags=re.IGNORECASE, na=False).sum()

339

In [228]:
# Select the budget values that don't match either forms, using Pandas element-wise logical operators.
matches_form_one = budget.str.contains(form_one, flags=re.IGNORECASE, na=False)
matches_form_two = budget.str.contains(form_two, flags=re.IGNORECASE, na=False)
budget[~matches_form_one & ~matches_form_two]

136                         Unknown
204     60 million Norwegian Kroner
478                         Unknown
973             $34 [3] [4] million
1126               $120 [4] million
1226                        Unknown
1278                            HBO
1374                     £6,000,000
1397                     13 million
1480                   £2.8 million
1734                   CAD2,000,000
1913     PHP 85 million (estimated)
1948                    102,888,900
1953                   3,500,000 DM
1973                     ₤2,300,874
2281                     $14 milion
2451                     ₤6,350,000
3144                   € 40 million
3360               $150 [6] million
3418                        $218.32
3802                   £4.2 million
3906                            N/A
3959                    760,000 USD
4470                       19 crore
4641                    £17 million
5034              $$200 [4] million
5055           $155 [2] [3] million
5419                $40 [4] 

In [229]:
len(budget[~matches_form_one & ~matches_form_two])

38

In [242]:
# Remove the citation references (numbers in square brackets), notice the need to escape brackets as special characters in regex.
# include regex=True to avoid warning message.
budget = budget.str.replace(r'\[\d+\]\s*','', regex=True)

In [243]:
# Check the remaing budget entries that don't match either forms. 
# Select the budget values that don't match either forms, using Pandas element-wise logical operators.
matches_form_one = budget.str.contains(form_one, flags=re.IGNORECASE, na=False)
matches_form_two = budget.str.contains(form_two, flags=re.IGNORECASE, na=False)
budget[~matches_form_one & ~matches_form_two]

136                         Unknown
204     60 million Norwegian Kroner
478                         Unknown
1226                        Unknown
1278                            HBO
1374                     £6,000,000
1397                     13 million
1480                   £2.8 million
1734                   CAD2,000,000
1913     PHP 85 million (estimated)
1948                    102,888,900
1953                   3,500,000 DM
1973                     ₤2,300,874
2281                     $14 milion
2451                     ₤6,350,000
3144                   € 40 million
3418                        $218.32
3802                   £4.2 million
3906                            N/A
3959                    760,000 USD
4470                       19 crore
4641                    £17 million
5424                            N/A
5447                     £4 million
5671                    €14 million
5687                   $ dead link]
6385                  £ 12 million 
6593                     £3 

In [244]:
len(budget[~matches_form_one & ~matches_form_two])

32

In [245]:
# now we have all we need to parse the budget values to numeric values. 
# extract the values from budget using str.extract. then apply the parse_dollars to the first col of DF returned by str.extract.
wiki_movies_df['budget'] = budget.str.extract(f'({form_one}|{form_two})', flags=re.IGNORECASE)[0].apply(parse_dollars)

In [246]:
# Check the new 'budget' column values in the DF.
wiki_movies_df['budget']

0       20000000.0
1        6000000.0
2       35000000.0
3       12000000.0
4       25000000.0
           ...    
7071    42000000.0
7072    60000000.0
7073    20000000.0
7074     9000000.0
7075           NaN
Name: budget, Length: 7033, dtype: float64

In [247]:
# Drop the 'Budget' column since it's no longer needed. 
wiki_movies_df.drop('Budget', axis=1, inplace=True)

In [248]:
# Check DF.
wiki_movies_df.head()

,url,year,imdb_link,title,Based on,Starring,Cinematography,Release date,Running time,Country,...,Director,Distributor,Editor(s),Composer(s),Producer(s),Production company(s),Writer(s),imdb_id,box_office,budget
0,https://en.wikipedia.org/wiki/The_Adventures_o...,1990,https://www.imdb.com/title/tt0098987/,The Adventures of Ford Fairlane,"[Characters, by Rex Weiner]","[Andrew Dice Clay, Wayne Newton, Priscilla Pre...",Oliver Wood,"[July 11, 1990, (, 1990-07-11, )]",102 minutes,United States,...,Renny Harlin,20th Century Fox,Michael Tronick,"[Cliff Eidelman, Yello]","[Steve Perry, Joel Silver]",Silver Pictures,"[David Arnott, James Cappe]",tt0098987,21400000.0,20000000.0
1,"https://en.wikipedia.org/wiki/After_Dark,_My_S...",1990,https://www.imdb.com/title/tt0098994/,"After Dark, My Sweet","[the novel, After Dark, My Sweet, by, Jim Thom...","[Jason Patric, Rachel Ward, Bruce Dern, George...",Mark Plummer,"[May 17, 1990, (, 1990-05-17, ), (Cannes Film ...",114 minutes,United States,...,James Foley,Avenue Pictures,Howard E. Smith,Maurice Jarre,"[Ric Kidney, Robert Redlin]",Avenue Pictures,"[James Foley, Robert Redlin]",tt0098994,2700000.0,6000000.0
2,https://en.wikipedia.org/wiki/Air_America_(film),1990,https://www.imdb.com/title/tt0099005/,Air America,"[Air America, by, Christopher Robbins]","[Mel Gibson, Robert Downey Jr., Nancy Travis, ...",Roger Deakins,"[August 10, 1990, (, 1990-08-10, )]",113 minutes,United States,...,Roger Spottiswoode,TriStar Pictures,"[John Bloom, Lois Freeman-Fox]",Charles Gross,Daniel Melnick,"[Carolco Pictures, IndieProd Company]","[John Eskow, Richard Rush]",tt0099005,57718089.0,35000000.0
3,https://en.wikipedia.org/wiki/Alice_(1990_film),1990,https://www.imdb.com/title/tt0099012/,Alice,NaN,"[Alec Baldwin, Blythe Danner, Judy Davis, Mia ...",Carlo Di Palma,"[December 25, 1990, (, 1990-12-25, )]",106 minutes,United States,...,Woody Allen,Orion Pictures,Susan E. Morse,NaN,Robert Greenhut,NaN,Woody Allen,tt0099012,7331647.0,12000000.0
4,https://en.wikipedia.org/wiki/Almost_an_Angel,1990,https://www.imdb.com/title/tt0099018/,Almost an Angel,NaN,"[Paul Hogan, Elias Koteas, Linda Kozlowski]",Russell Boyd,"December 19, 1990",95 minutes,US,...,John Cornell,Paramount Pictures,David Stiven,Maurice Jarre,John Cornell,NaN,Paul Hogan,tt0099018,6939946.0,25000000.0


In [249]:
# get the df columns. 
sorted(wiki_movies_df.columns.tolist())

['Based on',
 'Cinematography',
 'Composer(s)',
 'Country',
 'Director',
 'Distributor',
 'Editor(s)',
 'Language',
 'Producer(s)',
 'Production company(s)',
 'Release date',
 'Running time',
 'Starring',
 'Writer(s)',
 'box_office',
 'budget',
 'imdb_id',
 'imdb_link',
 'title',
 'url',
 'year']

### Parse Release Date in Wiki Data

In [250]:
# Collect the rows with non-null values release date data, it will be defined as a series, and convert lists to strings.
release_date = wiki_movies_df['Release date'].dropna().apply(lambda x: ' '.join(x) if type(x) == list else x)

In [251]:
# Check forms in release date series.
print(release_date.sample(n=5))

75         August 10, 1990 ( 1990-08-10 ) (United States)
575                        October 2, 1992 ( 1992-10-02 )
6414    July 30, 2015 ( 2015-07-30 ) ( L.A. Live premi...
6119                     February 14, 2014 ( 2014-02-14 )
6291    October 4, 2014 ( 2014-10-04 ) ( NYFF ) Decemb...
Name: Release date, dtype: object


In [ ]:
# # Build the pattern matching forms for release date data. 
# # e.g., form one= January 1, 2000; form two = 2000-01-01; form three = January 2000; form four = 2000.
# form_one = r"[a-zA-Z]+\s+\d+,\s+\d{4}"
# form_two = r"\d{4}(?:.*\d{2})+"
# form_three = r"[A-Za-z]+\s+\d{4}"
# form_four = r"\d{4}"

In [253]:
# Build the pattern matching forms for release date data. 
# e.g., form one= month, dd, yyyy; form two = yyyy-mm-dd or yyyy/mm/dd; form three = month yyyy; form four = yyyy.
date_form_one = r'(?:January|February|March|April|May|June|July|August|September|October|November|December)\s[123]?\d,\s\d{4}'
date_form_two = r'\d{4}.[01]\d.[0123]\d'
date_form_three = r'(?:January|February|March|April|May|June|July|August|September|October|November|December)\s\d{4}'
date_form_four = r'\d{4}'

In [254]:
# Extract the matching release dates.
release_date.str.extract(f'({date_form_one}|{date_form_two}|{date_form_three}|{date_form_four})', flags=re.IGNORECASE)

,0
0,"July 11, 1990"
1,"May 17, 1990"
2,"August 10, 1990"
3,"December 25, 1990"
4,"December 19, 1990"
...,...
7071,"December 25, 2018"
7072,"December 11, 2018"
7073,"November 8, 2018"
7074,"August 31, 2018"


In [255]:
# Parse the release date data using the pandas built-in to_datetime(), and create a new release_date col in the wiki_movies_df.
wiki_movies_df['release_date'] = pd.to_datetime(release_date.str.extract(f'({date_form_one}|{date_form_two}|{date_form_three}|{date_form_four})')[0], infer_datetime_format=True)

In [256]:
# Check the DF.
wiki_movies_df.head()

,url,year,imdb_link,title,Based on,Starring,Cinematography,Release date,Running time,Country,...,Distributor,Editor(s),Composer(s),Producer(s),Production company(s),Writer(s),imdb_id,box_office,budget,release_date
0,https://en.wikipedia.org/wiki/The_Adventures_o...,1990,https://www.imdb.com/title/tt0098987/,The Adventures of Ford Fairlane,"[Characters, by Rex Weiner]","[Andrew Dice Clay, Wayne Newton, Priscilla Pre...",Oliver Wood,"[July 11, 1990, (, 1990-07-11, )]",102 minutes,United States,...,20th Century Fox,Michael Tronick,"[Cliff Eidelman, Yello]","[Steve Perry, Joel Silver]",Silver Pictures,"[David Arnott, James Cappe]",tt0098987,21400000.0,20000000.0,1990-07-11
1,"https://en.wikipedia.org/wiki/After_Dark,_My_S...",1990,https://www.imdb.com/title/tt0098994/,"After Dark, My Sweet","[the novel, After Dark, My Sweet, by, Jim Thom...","[Jason Patric, Rachel Ward, Bruce Dern, George...",Mark Plummer,"[May 17, 1990, (, 1990-05-17, ), (Cannes Film ...",114 minutes,United States,...,Avenue Pictures,Howard E. Smith,Maurice Jarre,"[Ric Kidney, Robert Redlin]",Avenue Pictures,"[James Foley, Robert Redlin]",tt0098994,2700000.0,6000000.0,1990-05-17
2,https://en.wikipedia.org/wiki/Air_America_(film),1990,https://www.imdb.com/title/tt0099005/,Air America,"[Air America, by, Christopher Robbins]","[Mel Gibson, Robert Downey Jr., Nancy Travis, ...",Roger Deakins,"[August 10, 1990, (, 1990-08-10, )]",113 minutes,United States,...,TriStar Pictures,"[John Bloom, Lois Freeman-Fox]",Charles Gross,Daniel Melnick,"[Carolco Pictures, IndieProd Company]","[John Eskow, Richard Rush]",tt0099005,57718089.0,35000000.0,1990-08-10
3,https://en.wikipedia.org/wiki/Alice_(1990_film),1990,https://www.imdb.com/title/tt0099012/,Alice,NaN,"[Alec Baldwin, Blythe Danner, Judy Davis, Mia ...",Carlo Di Palma,"[December 25, 1990, (, 1990-12-25, )]",106 minutes,United States,...,Orion Pictures,Susan E. Morse,NaN,Robert Greenhut,NaN,Woody Allen,tt0099012,7331647.0,12000000.0,1990-12-25
4,https://en.wikipedia.org/wiki/Almost_an_Angel,1990,https://www.imdb.com/title/tt0099018/,Almost an Angel,NaN,"[Paul Hogan, Elias Koteas, Linda Kozlowski]",Russell Boyd,"December 19, 1990",95 minutes,US,...,Paramount Pictures,David Stiven,Maurice Jarre,John Cornell,NaN,Paul Hogan,tt0099018,6939946.0,25000000.0,1990-12-19


In [257]:
wiki_movies_df['release_date'].isnull().sum()

32

In [262]:
wiki_movies_df.loc[wiki_movies_df['release_date'].isnull()]

,url,year,imdb_link,title,Based on,Starring,Cinematography,Release date,Running time,Country,...,Distributor,Editor(s),Composer(s),Producer(s),Production company(s),Writer(s),imdb_id,box_office,budget,release_date
212,https://en.wikipedia.org/wiki/Snake_Eater_II:_...,1990,https://www.imdb.com/title/tt0102950/,Snake Eater II: The Drug Buster,NaN,"[Lorenzo Lamas, Ron Palillo]",Glen MacPherson,NaN,93 minutes,Canada,...,NaN,NaN,John Massari,NaN,NaN,"[Don Carmody, John Dunning, Michael Paseornek]",tt0102950,NaN,NaN,NaT
1044,"https://en.wikipedia.org/wiki/No_Dessert,_Dad,...",1994,https://www.imdb.com/title/tt0110676/,"No Dessert, Dad, till You Mow the Lawn",NaN,"[Robert Hays, Joanna Kerns]",NaN,NaN,89 min.,NaN,...,NaN,NaN,NaN,NaN,NaN,"[Jennifer Moran,, Martha Moran]",tt0110676,NaN,NaN,NaT
1173,https://en.wikipedia.org/wiki/Black_Scorpion_(...,1995,https://www.imdb.com/title/tt0112519/,Black Scorpion,NaN,"[Joan Severance, Bruce Abbott, Garrett Morris]",Geoff George,NaN,90 min.,United States,...,Showtime,"[Tom Petersen, Gwyneth Gibby]",Kevin Kiner,Mike Elliott,New Horizons,Craig J. Nevius,tt0112519,NaN,NaN,NaT
1234,https://en.wikipedia.org/wiki/Flirt_(1995_film),1995,https://www.imdb.com/title/tt0113080/,Flirt,NaN,"[Bill Sage, Dwight Ewell, Miho Nikaido]",Michael Spiller,NaN,83 minutes,"[United States, Germany, Japan, [1]]",...,Pandora Film (Germany),"[Hal Hartley, Steve Hamilton, [1]]","[Hal Hartley, Jeffrey Taylor, [1]]",Ted Hope,Good Machine,Hal Hartley,tt0113080,NaN,NaN,NaT
1302,https://en.wikipedia.org/wiki/Let_It_Be_Me_(19...,1995,https://www.imdb.com/title/tt0113638/,Let It Be Me,NaN,NaN,Miroslav Ondrícek,NaN,NaN,NaN,...,NaN,"[Patricia Bowers, Ray Hubley, Bill Pankow]",NaN,NaN,NaN,Eleanor Bergstein,tt0113638,NaN,NaN,NaT
1346,https://en.wikipedia.org/wiki/No_Exit_(1995_film),1995,https://www.imdb.com/title/tt0113990/,No Exit,NaN,"[Jeff Wincott, Sven-Ole Thorsen]",Russ Goozee,NaN,96 minutes,Canada,...,No Exit Productions Limited,David Ransley,Ken Greer,NaN,NaN,"[John Lawson, Damian Lee]",tt0113990,NaN,NaN,NaT
1635,https://en.wikipedia.org/wiki/Listen_(film),1996,https://www.imdb.com/title/tt2910992/,Listen,NaN,"[Joshua Mikel, Julie Moss, Avery Pohl, Julie M...",F.C.Rabbath,NaN,90 Minutes,United States,...,NaN,NaN,"[Tony Anderson, Sean Beeson, Valentin Boomes, ...","[F.C.Rabbath, Vanessa Vandy, Adam Bertocci]",F.C.Rabbath Creations,F.C.Rabbath,tt2910992,NaN,NaN,NaT
1644,https://en.wikipedia.org/wiki/Magenta_(film),1996,https://www.imdb.com/title/tt0116956/,Magenta,NaN,"[Julian McMahon, Alison Storry, Marklen Kenned...",Mark Anthony Galluzzo,NaN,91 minutes,United States,...,NaN,Paul Osborne,"[Harald Kloser, Thomas Schobel (co-composer)]",Bill and Michael Talerico,NaN,Gregory C. Haynes,tt0116956,NaN,NaN,NaT
1777,https://en.wikipedia.org/wiki/Your_Studio_and_You,1996,https://www.imdb.com/title/tt0159144/,Your Studio and You,NaN,"[Sylvester Stallone, Demi Moore, Steven Spielb...",Kenny Gioseffi,NaN,approx. 14 min,United States,...,Universal Studios,NaN,NaN,NaN,NaN,"[Trey Parker, Matt Stone]",tt0159144,NaN,NaN,NaT
1836,https://en.wikipedia.org/wiki/A_Brooklyn_State...,1997,https://www.imdb.com/title/tt0118782/,A Brooklyn State of Mind,NaN,NaN,Ken Kelsch,NaN,NaN,NaN,...,NaN,NaN,Paul Zaza,NaN,NaN,NaN,tt0118782,NaN,NaN,NaT


In [478]:
# Drop the original 'Release date' column from the DF. 
wiki_movies_df.drop('Release date', axis=1, inplace=True)

In [479]:
wiki_movies_df.head()

,url,year,imdb_link,title,Based on,Starring,Cinematography,Running time,Country,Language,...,Editor(s),Composer(s),Producer(s),Production company(s),Writer(s),imdb_id,box_office,budget,release_date,running_time
0,https://en.wikipedia.org/wiki/The_Adventures_o...,1990,https://www.imdb.com/title/tt0098987/,The Adventures of Ford Fairlane,"[Characters, by Rex Weiner]","[Andrew Dice Clay, Wayne Newton, Priscilla Pre...",Oliver Wood,102 minutes,United States,English,...,Michael Tronick,"[Cliff Eidelman, Yello]","[Steve Perry, Joel Silver]",Silver Pictures,"[David Arnott, James Cappe]",tt0098987,21400000.0,20000000.0,1990-07-11,102.0
1,"https://en.wikipedia.org/wiki/After_Dark,_My_S...",1990,https://www.imdb.com/title/tt0098994/,"After Dark, My Sweet","[the novel, After Dark, My Sweet, by, Jim Thom...","[Jason Patric, Rachel Ward, Bruce Dern, George...",Mark Plummer,114 minutes,United States,English,...,Howard E. Smith,Maurice Jarre,"[Ric Kidney, Robert Redlin]",Avenue Pictures,"[James Foley, Robert Redlin]",tt0098994,2700000.0,6000000.0,1990-05-17,114.0
2,https://en.wikipedia.org/wiki/Air_America_(film),1990,https://www.imdb.com/title/tt0099005/,Air America,"[Air America, by, Christopher Robbins]","[Mel Gibson, Robert Downey Jr., Nancy Travis, ...",Roger Deakins,113 minutes,United States,"[English, Lao]",...,"[John Bloom, Lois Freeman-Fox]",Charles Gross,Daniel Melnick,"[Carolco Pictures, IndieProd Company]","[John Eskow, Richard Rush]",tt0099005,57718089.0,35000000.0,1990-08-10,113.0
3,https://en.wikipedia.org/wiki/Alice_(1990_film),1990,https://www.imdb.com/title/tt0099012/,Alice,NaN,"[Alec Baldwin, Blythe Danner, Judy Davis, Mia ...",Carlo Di Palma,106 minutes,United States,English,...,Susan E. Morse,NaN,Robert Greenhut,NaN,Woody Allen,tt0099012,7331647.0,12000000.0,1990-12-25,106.0
4,https://en.wikipedia.org/wiki/Almost_an_Angel,1990,https://www.imdb.com/title/tt0099018/,Almost an Angel,NaN,"[Paul Hogan, Elias Koteas, Linda Kozlowski]",Russell Boyd,95 minutes,US,English,...,David Stiven,Maurice Jarre,John Cornell,NaN,Paul Hogan,tt0099018,6939946.0,25000000.0,1990-12-19,95.0


### Parse Running Time in Wiki Data

In [263]:
# Collect the rows with non-null values of running time data, it will be defined as a series, and convert lists to strings.
running_time = wiki_movies_df['Running time'].dropna().apply(lambda x: ' '.join(x) if type(x) == list else x)

In [297]:
len(running_time)

6894

In [294]:
running_time.sample(n=20)

3015     91 minutes
4066        100 min
2549    114 minutes
5661     92 minutes
2040     92 minutes
856      95 minutes
5460    120 minutes
669     126 minutes
2996     96 minutes
2222     98 minutes
1156        87 min.
5190    123 minutes
3617     86 minutes
1673     95 minutes
2543     96 minutes
6191    134 minutes
5537    165 minutes
3808    144 minutes
4398     90 minutes
402     132 minutes
Name: Running time, dtype: object

In [295]:
# Find out how many running times look exactly like '100 minutes' by using string boundaries. 
running_time.str.contains(r'^\d*\s*minutes$', flags=re.IGNORECASE, na=False).sum()

6528

In [296]:
# Find out what the remaining (366) running times look like. 
running_time[running_time.str.contains(r'^\d*\s*minutes$', flags=re.IGNORECASE, na=False) != True]

9                                                 102 min
26                                                 93 min
28                                                32 min.
34                                                101 min
35                                                 97 min
                              ...                        
6500       114 minutes [1] 120 minutes (extended edition)
6643                                             104 mins
6709    90 minutes (theatrical) [1] 91 minutes (unrate...
7057    108 minutes (Original cut) 98 minutes (UK cut)...
7075                Variable; 90 minutes for default path
Name: Running time, Length: 366, dtype: object

In [298]:
# Make the pattern match form more general 
# by only marking the beginning of the string and accepting other abb. of 'minutes' by only searching up to the letter 'm'.
running_time.str.contains(r'^\d*\s*m', flags=re.IGNORECASE, na=False).sum()

6877

In [299]:
# Get the remaining (17) entries that don't match the general pattern. 
running_time[running_time.str.contains(r'^\d*\s*m', flags=re.IGNORECASE, na=False) != True]

668                     UK:84 min (DVD version) US:86 min
727                         78-102 min (depending on cut)
840                       Varies (79 [3] –84 [1] minutes)
1347                                              25 : 03
1443    United States: 77 minutes Argentina: 94 minute...
1499                                            1hr 35min
1551                                               varies
1774                    Netherlands:96 min, Canada:95 min
1777                                       approx. 14 min
2273                                           1 h 43 min
2993                                               1h 48m
3925                                              4 hours
4425    US domestic version: 86 minutes Original versi...
4967    Theatrical cut: 97 minutes Unrated cut: 107 mi...
5424                    115 [1] /123 [2] /128 [3] minutes
5447                                    1 hour 32 minutes
7075                Variable; 90 minutes for default path
Name: Running 

In [300]:
# Create a form that matches the hour + minute pattern witn one regex.
hour_min_form = r'\d+\s*ho?u?r?s?\s*\d*'

In [433]:
# Extract the running time data, only capture the digits, allowing for both possible patterns,
# Relax the first minutes form of how that pattern should start at the beginning of the string by removing ^.
running_time_extract = running_time.str.extract(r'(\d+)\s*ho?u?r?s?\s*(\d*)|(\d+)\s*m')

In [434]:
# Check the 3 digits groups captured from running time data. 
running_time_extract

,0,1,2
0,NaN,NaN,102
1,NaN,NaN,114
2,NaN,NaN,113
3,NaN,NaN,106
4,NaN,NaN,95
...,...,...,...
7071,NaN,NaN,90
7072,NaN,NaN,132
7073,NaN,NaN,120
7074,NaN,NaN,123


In [459]:
running_time_extract.sample(n=5)

,0,1,2
3494,NaN,NaN,98
2187,NaN,NaN,124
3424,NaN,NaN,104
912,NaN,NaN,48
3987,NaN,NaN,107


In [424]:
running_time

0                                 102 minutes
1                                 114 minutes
2                                 113 minutes
3                                 106 minutes
4                                  95 minutes
                        ...                  
7071                               90 minutes
7072                              132 minutes
7073                              120 minutes
7074                              123 minutes
7075    Variable; 90 minutes for default path
Name: Running time, Length: 6894, dtype: object

In [436]:
len(running_time_extract)

6894

In [437]:
running_time_extract.iloc[-5:]

,0,1,2
7071,NaN,NaN,90
7072,NaN,NaN,132
7073,NaN,NaN,120
7074,NaN,NaN,123
7075,NaN,NaN,90


In [465]:
running_time_extract.loc[1499]

0      1
1     35
2    NaN
Name: 1499, dtype: object

In [471]:
running_time_extract.loc[running_time_extract[0].notnull() == True]

,0,1,2
1499,1,35,NaN
2273,1,43,NaN
2993,1,48,NaN
3925,4,,NaN
5447,1,32,NaN


In [472]:
# form catpured only the first occurence of time in the entry.
running_time_extract.loc[1774]

0    NaN
1    NaN
2     96
Name: 1774, dtype: object

In [473]:
# Convert running_time string entries to numeric values, coerce errors to turn empty strings to NaN, then change NaNs to zeros.
running_time_extract = running_time_extract.apply(lambda col: pd.to_numeric(col, errors='coerce')).fillna(0)

In [474]:
# Check results
running_time_extract

,0,1,2
0,0.0,0.0,102.0
1,0.0,0.0,114.0
2,0.0,0.0,113.0
3,0.0,0.0,106.0
4,0.0,0.0,95.0
...,...,...,...
7071,0.0,0.0,90.0
7072,0.0,0.0,132.0
7073,0.0,0.0,120.0
7074,0.0,0.0,123.0


In [475]:
# Convert the hour and the minute capture groups to minutes if the pure minutes capture group is zero.
# Save the output to wiki_movies_df.
wiki_movies_df['running_time'] = running_time_extract.apply(lambda row: row[0]*60 + row[1] if row[2] == 0 else row[2], axis=1)

In [476]:
# Check the new 'running_time' col in df.
wiki_movies_df.head()

,url,year,imdb_link,title,Based on,Starring,Cinematography,Release date,Running time,Country,...,Editor(s),Composer(s),Producer(s),Production company(s),Writer(s),imdb_id,box_office,budget,release_date,running_time
0,https://en.wikipedia.org/wiki/The_Adventures_o...,1990,https://www.imdb.com/title/tt0098987/,The Adventures of Ford Fairlane,"[Characters, by Rex Weiner]","[Andrew Dice Clay, Wayne Newton, Priscilla Pre...",Oliver Wood,"[July 11, 1990, (, 1990-07-11, )]",102 minutes,United States,...,Michael Tronick,"[Cliff Eidelman, Yello]","[Steve Perry, Joel Silver]",Silver Pictures,"[David Arnott, James Cappe]",tt0098987,21400000.0,20000000.0,1990-07-11,102.0
1,"https://en.wikipedia.org/wiki/After_Dark,_My_S...",1990,https://www.imdb.com/title/tt0098994/,"After Dark, My Sweet","[the novel, After Dark, My Sweet, by, Jim Thom...","[Jason Patric, Rachel Ward, Bruce Dern, George...",Mark Plummer,"[May 17, 1990, (, 1990-05-17, ), (Cannes Film ...",114 minutes,United States,...,Howard E. Smith,Maurice Jarre,"[Ric Kidney, Robert Redlin]",Avenue Pictures,"[James Foley, Robert Redlin]",tt0098994,2700000.0,6000000.0,1990-05-17,114.0
2,https://en.wikipedia.org/wiki/Air_America_(film),1990,https://www.imdb.com/title/tt0099005/,Air America,"[Air America, by, Christopher Robbins]","[Mel Gibson, Robert Downey Jr., Nancy Travis, ...",Roger Deakins,"[August 10, 1990, (, 1990-08-10, )]",113 minutes,United States,...,"[John Bloom, Lois Freeman-Fox]",Charles Gross,Daniel Melnick,"[Carolco Pictures, IndieProd Company]","[John Eskow, Richard Rush]",tt0099005,57718089.0,35000000.0,1990-08-10,113.0
3,https://en.wikipedia.org/wiki/Alice_(1990_film),1990,https://www.imdb.com/title/tt0099012/,Alice,NaN,"[Alec Baldwin, Blythe Danner, Judy Davis, Mia ...",Carlo Di Palma,"[December 25, 1990, (, 1990-12-25, )]",106 minutes,United States,...,Susan E. Morse,NaN,Robert Greenhut,NaN,Woody Allen,tt0099012,7331647.0,12000000.0,1990-12-25,106.0
4,https://en.wikipedia.org/wiki/Almost_an_Angel,1990,https://www.imdb.com/title/tt0099018/,Almost an Angel,NaN,"[Paul Hogan, Elias Koteas, Linda Kozlowski]",Russell Boyd,"December 19, 1990",95 minutes,US,...,David Stiven,Maurice Jarre,John Cornell,NaN,Paul Hogan,tt0099018,6939946.0,25000000.0,1990-12-19,95.0


In [477]:
# Check the success of the conversion on one of the entries with hour, minute running time.
wiki_movies_df.loc[1499]

url                      https://en.wikipedia.org/wiki/Christmas_Every_Day
year                                                                  1996
imdb_link                            https://www.imdb.com/title/tt0115891/
title                                                  Christmas Every Day
Based on                                                               NaN
Starring                 [Erik von Detten, Bess Armstrong, Robert Hays,...
Cinematography                                               Gideon Porath
Release date                     [December 1, 1996, (, 1996-12-01, ), [1]]
Running time                                                     1hr 35min
Country                                                      United States
Language                                                               NaN
Director                                                      Larry Peerce
Distributor                                 Disney-ABC Domestic Television
Editor(s)                

In [480]:
# Drop original 'Running time' column from the dataset. 
wiki_movies_df.drop('Running time', axis=1, inplace=True)

In [481]:
# Check the completely clean Wikipedia dataset.
wiki_movies_df.head()

,url,year,imdb_link,title,Based on,Starring,Cinematography,Country,Language,Director,...,Editor(s),Composer(s),Producer(s),Production company(s),Writer(s),imdb_id,box_office,budget,release_date,running_time
0,https://en.wikipedia.org/wiki/The_Adventures_o...,1990,https://www.imdb.com/title/tt0098987/,The Adventures of Ford Fairlane,"[Characters, by Rex Weiner]","[Andrew Dice Clay, Wayne Newton, Priscilla Pre...",Oliver Wood,United States,English,Renny Harlin,...,Michael Tronick,"[Cliff Eidelman, Yello]","[Steve Perry, Joel Silver]",Silver Pictures,"[David Arnott, James Cappe]",tt0098987,21400000.0,20000000.0,1990-07-11,102.0
1,"https://en.wikipedia.org/wiki/After_Dark,_My_S...",1990,https://www.imdb.com/title/tt0098994/,"After Dark, My Sweet","[the novel, After Dark, My Sweet, by, Jim Thom...","[Jason Patric, Rachel Ward, Bruce Dern, George...",Mark Plummer,United States,English,James Foley,...,Howard E. Smith,Maurice Jarre,"[Ric Kidney, Robert Redlin]",Avenue Pictures,"[James Foley, Robert Redlin]",tt0098994,2700000.0,6000000.0,1990-05-17,114.0
2,https://en.wikipedia.org/wiki/Air_America_(film),1990,https://www.imdb.com/title/tt0099005/,Air America,"[Air America, by, Christopher Robbins]","[Mel Gibson, Robert Downey Jr., Nancy Travis, ...",Roger Deakins,United States,"[English, Lao]",Roger Spottiswoode,...,"[John Bloom, Lois Freeman-Fox]",Charles Gross,Daniel Melnick,"[Carolco Pictures, IndieProd Company]","[John Eskow, Richard Rush]",tt0099005,57718089.0,35000000.0,1990-08-10,113.0
3,https://en.wikipedia.org/wiki/Alice_(1990_film),1990,https://www.imdb.com/title/tt0099012/,Alice,NaN,"[Alec Baldwin, Blythe Danner, Judy Davis, Mia ...",Carlo Di Palma,United States,English,Woody Allen,...,Susan E. Morse,NaN,Robert Greenhut,NaN,Woody Allen,tt0099012,7331647.0,12000000.0,1990-12-25,106.0
4,https://en.wikipedia.org/wiki/Almost_an_Angel,1990,https://www.imdb.com/title/tt0099018/,Almost an Angel,NaN,"[Paul Hogan, Elias Koteas, Linda Kozlowski]",Russell Boyd,US,English,John Cornell,...,David Stiven,Maurice Jarre,John Cornell,NaN,Paul Hogan,tt0099018,6939946.0,25000000.0,1990-12-19,95.0


In [482]:
# Get the clean columns. 
sorted(wiki_movies_df.columns.tolist())

['Based on',
 'Cinematography',
 'Composer(s)',
 'Country',
 'Director',
 'Distributor',
 'Editor(s)',
 'Language',
 'Producer(s)',
 'Production company(s)',
 'Starring',
 'Writer(s)',
 'box_office',
 'budget',
 'imdb_id',
 'imdb_link',
 'release_date',
 'running_time',
 'title',
 'url',
 'year']